**Edge-IIoTset Dataset Exploration and Preprocessing**

In [5]:
import itertools  # 高效循环的迭代器函数
import numpy as np
import pandas as pd
import seaborn as sns # seaborn是一个基于matplotlib的Python数据可视化库,使得创建复杂的统计图表变得更加容易和美观; 与pandas数据结构紧密集成
import matplotlib.pyplot as plt

sns.set() # set seaborn to default settings 将seaborn设置为默认模式

**Import the data into a pandas data-frame and inspect it’s properties.**  
**导入数据并查看属性**

In [ ]:
df = pd.read_csv('DNN-EdgeIIoT-dataset.csv', low_memory=False)
df.shape # (2219201, 63)
df.head()

**Result:the data is in tabular form, with 2219201 entries, each having 63 distinct attributes.**

**Information About Data**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2219201 entries, 0 to 2219200
Data columns (total 63 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   frame.time                 object 
 1   ip.src_host                object 
 2   ip.dst_host                object 
 3   arp.dst.proto_ipv4         object 
 4   arp.opcode                 float64
 5   arp.hw.size                float64
 6   arp.src.proto_ipv4         object 
 7   icmp.checksum              float64
 8   icmp.seq_le                float64
 9   icmp.transmit_timestamp    float64
 10  icmp.unused                float64
 11  http.file_data             object 
 12  http.content_length        float64
 13  http.request.uri.query     object 
 14  http.request.method        object 
 15  http.referer               object 
 16  http.request.full_uri      object 
 17  http.request.version       object 
 18  http.response              float64
 19  http.tls_port              float64
 20  tc

**Let’s see how many unique values are there in each column.**

In [ ]:
df.nunique(axis=0) # 每列有多少个独一值

,0
frame.time,2206364
ip.src_host,137167
ip.dst_host,52425
arp.dst.proto_ipv4,9
arp.opcode,3
...,...
mbtcp.len,8
mbtcp.trans_id,151
mbtcp.unit_id,6
Attack_label,2


**Now, inspect for NaNs in the data.**
检查NaN值

In [ ]:
df.isna().sum()

,0
frame.time,0
ip.src_host,0
ip.dst_host,0
arp.dst.proto_ipv4,0
arp.opcode,0
...,...
mbtcp.len,0
mbtcp.trans_id,0
mbtcp.unit_id,0
Attack_label,0


In [ ]:
df.isnull().values.any()   #No null values in dataset
# 用于检查数组中是否存在任何True值  有True返回Ture；否则返回False
#也就是说 df纯在NaN就转为True，否则为False；然后输出为Numpy数组，再判断改数组是否存在True

False

**Let’s see how many duplicate entries are there and drop them if there are any.**处理重复值

In [ ]:
# find total duplicate entries and drop them if any
print(f'Total duplicate rows: {df.duplicated().sum()}')

Total duplicate rows: 815


In [ ]:
# drop duplicate rows if any
df = df[~df.duplicated()]

In [ ]:
df.shape  # 删掉重复值后的数据规模

(2218386, 63)

In [ ]:
print(f'total duplicate rows: {df.duplicated().sum()}') # 再次查看是否存在重复值

total duplicate rows: 0


In [ ]:
df.head()

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.request.method,http.referer,http.request.version,http.response,...,mqtt.msgtype,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_type
1090368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,Normal
251032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,Normal
1724312,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_HTTP
943694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,Normal
1185879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,Normal


**Remove insignificant columns from data frame**

In [ ]:
from sklearn.utils import shuffle # 用于随机打乱数据集中的样本顺序
# 用list删掉列
drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4","arp.dst.proto_ipv4",
                "http.file_data","http.request.full_uri","icmp.transmit_timestamp",
                "http.request.uri.query", "tcp.options","tcp.payload","tcp.srcport",
                "tcp.dstport", "udp.port", "mqtt.msg", "Attack_label"]

df.drop(drop_columns, axis=1, inplace=True)
df.dropna(axis=0, how='any', inplace=True)
df.drop_duplicates(subset=None, keep="first", inplace=True)
df = shuffle(df)
df.isna().sum() # 每一列的缺失值数量
df.head()

In [ ]:
df.shape

(1909671, 47)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1909671 entries, 1090368 to 747184
Data columns (total 47 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   arp.opcode                 float64
 1   arp.hw.size                float64
 2   icmp.checksum              float64
 3   icmp.seq_le                float64
 4   icmp.unused                float64
 5   http.content_length        float64
 6   http.request.method        object 
 7   http.referer               object 
 8   http.request.version       object 
 9   http.response              float64
 10  http.tls_port              float64
 11  tcp.ack                    float64
 12  tcp.ack_raw                float64
 13  tcp.checksum               float64
 14  tcp.connection.fin         float64
 15  tcp.connection.rst         float64
 16  tcp.connection.syn         float64
 17  tcp.connection.synack      float64
 18  tcp.flags                  float64
 19  tcp.flags.ack              float64
 20  tc

**Visulalize the colums with objects.**

**http、dns、mqtt一共7个特征**

In [ ]:
df['http.request.method'].value_counts()

,count
http.request.method,
0.0,1658469
0,219116
GET,30473
POST,1212
TRACE,387
OPTIONS,6
PROPFIND,4
PUT,2
SEARCH,2


In [ ]:
df['http.referer'].value_counts()

,count
http.referer,
0.0,1796035
0,113066
127.0.0.1,383
() { _; } >_[$($())] { echo 93e4r0-CVE-2014-6278: true; echo;echo; },185
TESTING_PURPOSES_ONLY,2


In [ ]:
df['http.request.version'].value_counts()

,count
http.request.version,
0.0,1658111
0,219474
HTTP/1.1,28681
HTTP/1.0,3382
-a HTTP/1.1,4
"Src=javascript:alert('Vulnerable')><Img Src=\"" HTTP/1.1",4
-al&ABSOLUTE_PATH_STUDIP=http://cirt.net/rfiinc.txt?? HTTP/1.1,3
"script>alert(1)/script><\"" HTTP/1.1",3
> HTTP/1.1,2


In [ ]:
df['dns.qry.name.len'].value_counts()

,count
dns.qry.name.len,
0,1362163
0.0,545446
0.debian.pool.ntp.org,450
3.debian.pool.ntp.org,448
2.debian.pool.ntp.org,444
1.debian.pool.ntp.org,443
1.0,227
raspberrypi.local,38
null-null.local,6


In [ ]:
df['mqtt.conack.flags'].value_counts()

,count
mqtt.conack.flags,
0,1280941
0.0,545673
0x00000000,83017
1574359,9
1574358,9
1461383,4
1461384,4
1461589,3
1461591,3


In [ ]:
df['mqtt.protoname'].value_counts()

,count
mqtt.protoname,
0,1280980
0.0,545673
MQTT,83018


In [ ]:
df['mqtt.topic'].value_counts()

,count
mqtt.topic,
0,1280983
0.0,545673
Temperature_and_Humidity,83015


**Apply Dummy Encoding for Text应用虚拟编码**

In [ ]:
# 通过创建二进制来标识分量的类别，1表示类别存在,0表示类别不存在
import numpy as np
from sklearn.model_selection import train_test_split # 用于数据集的划分
from sklearn.preprocessing import StandardScaler # 用于数据预处理
from sklearn import preprocessing

# 函数将指定的分类变量转换为虚拟编码，并将结果添加到原始DataFrame中，然后删除原始的分类变量列
def encode_text_dummy(df, name):

    dummies = pd.get_dummies(df[name])  # 对指定列进行虚拟编码,生成一个新的DataFrame

    for x in dummies.columns: # 每个类别创建一个新的列，列名为 {name}-{类别}

        dummy_name = f"{name}-{x}"

        df[dummy_name] = dummies[x] # 将虚拟编码的结果添加到原始DataFrame中

    df.drop(name, axis=1, inplace=True) # 删除原始的分类变量列

# 对需要虚拟编码的特征进行函数调用
encode_text_dummy(df,'http.request.method')
encode_text_dummy(df,'http.referer')
encode_text_dummy(df,"http.request.version")
encode_text_dummy(df,"dns.qry.name.len")
encode_text_dummy(df,"mqtt.conack.flags")
encode_text_dummy(df,"mqtt.protoname")
encode_text_dummy(df,"mqtt.topic")
# 简单的来说  就是利用扩张列的方法,对特征进行虚拟编码

In [ ]:
df.head()

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.response,http.tls_port,tcp.ack,tcp.ack_raw,...,mqtt.conack.flags-1471198,mqtt.conack.flags-1471199,mqtt.conack.flags-1574358,mqtt.conack.flags-1574359,mqtt.protoname-0,mqtt.protoname-0.0,mqtt.protoname-MQTT,mqtt.topic-0,mqtt.topic-0.0,mqtt.topic-Temperature_and_Humidity
1090368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.292691e+09,...,False,False,False,False,True,False,False,True,False,False
251032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,2.210188e+09,...,False,False,False,False,True,False,False,True,False,False
1724312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.010767e+06,...,False,False,False,False,False,True,False,False,True,False
943694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.536111e+08,...,False,False,False,False,True,False,False,True,False,False
1185879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,3.854373e+09,...,False,False,False,False,True,False,False,True,False,False


In [ ]:
df.shape  # 可以看出  应用后的列变多了

(1909671, 96)

**Move Attack_Type at the End**

In [ ]:
# 将攻击类别放到最后一列
# 首先获取DataFrame的所有列名，然后删除Attack_type列，最后将Attack_type列添加到列列表的末尾，并重新排列DataFrame的列顺序。
cols = list(df.columns.values)
cols.pop(cols.index('Attack_type'))
df = df[cols+['Attack_type']]

In [ ]:
df.head()

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.response,http.tls_port,tcp.ack,tcp.ack_raw,...,mqtt.conack.flags-1471199,mqtt.conack.flags-1574358,mqtt.conack.flags-1574359,mqtt.protoname-0,mqtt.protoname-0.0,mqtt.protoname-MQTT,mqtt.topic-0,mqtt.topic-0.0,mqtt.topic-Temperature_and_Humidity,Attack_type
1090368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.292691e+09,...,False,False,False,True,False,False,True,False,False,Normal
251032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,2.210188e+09,...,False,False,False,True,False,False,True,False,False,Normal
1724312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.010767e+06,...,False,False,False,False,True,False,False,True,False,DDoS_HTTP
943694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.536111e+08,...,False,False,False,True,False,False,True,False,False,Normal
1185879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,3.854373e+09,...,False,False,False,True,False,False,True,False,False,Normal


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1909671 entries, 1090368 to 747184
Data columns (total 96 columns):
 #   Column                                                                                                Dtype  
---  ------                                                                                                -----  
 0   arp.opcode                                                                                            float64
 1   arp.hw.size                                                                                           float64
 2   icmp.checksum                                                                                         float64
 3   icmp.seq_le                                                                                           float64
 4   icmp.unused                                                                                           float64
 5   http.content_length                                                          

**Export Complete Preprocessed Dataset**

In [ ]:
df.to_csv('processed_IIoT.csv', encoding='utf-8', index=False)